<a href="https://colab.research.google.com/github/AmanullaChishty/Pytorch/blob/main/Multimodel_Experiment_tracking_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ways to track ML experimnets
 - TensorBoard
 - Weights & Biases Experiment Tracking
 - MLFlow

## 0. Setup

In [17]:
import torch
import torchvision

In [18]:
import matplotlib.pyplot as plt
import numpy as np

from torch import nn
from torchvision import transforms

In [19]:
!pip install -q torchinfo
from torchinfo import summary

In [20]:
!git clone https://github.com/mrdbourke/pytorch-deep-learning
!mv pytorch-deep-learning/going_modular .
!rm -rf pytorch-deep-learning


Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4056, done.
remote: Total 4056 (delta 0), reused 0 (delta 0), pack-reused 4056
Receiving objects: 100% (4056/4056), 646.90 MiB | 21.31 MiB/s, done.
Resolving deltas: 100% (2371/2371), done.
Updating files: 100% (248/248), done.
mv: cannot move 'pytorch-deep-learning/going_modular' to './going_modular': Directory not empty


In [21]:
from going_modular.going_modular import data_setup, engine

In [22]:
def set_seeds(seed: int=42):
  torch.manual_seed(seed)

## 1. Get Data

In [23]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source:str,
                  destination:str,
                  remove_source: bool = True) -> Path:
  data_path = Path("data/")
  image_path = data_path / destination

  if image_path.is_dir():
    print(f"[INFO] {image_path} directory exists, skipping download.")
  else:
    print(f"[INFO] Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

    target_file = Path(source).name
    with open(data_path/ target_file, "wb") as f:
      request = requests.get(source)
      print(f"[INFO] Downloading {target_file} from {source}...")
      f.write(request.content)

    with zipfile.ZipFile(data_path/target_file,"r") as zip_ref:
      print(f"[INFO] Unzipping {target_file} data...")
      zip_ref.extractall(image_path)

    if remove_source:
      os.remove(data_path/target_file)
  return image_path

In [24]:
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] data/pizza_steak_sushi directory exists, skipping download.


PosixPath('data/pizza_steak_sushi')

## 2. Create Datasets and Dataloaders

In [25]:
## Normalize in ImageNet format
normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                 std = [0.229, 0.224, 0.225])

In [26]:
train_dir = image_path / "train"
test_dir = image_path / "test"

train_transforms = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  normalize
])
print(f"Manually created transforms: {train_transforms}")

train_datalaoder, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=train_transforms,
    batch_size=32
)

train_datalaoder, test_dataloader, class_names

Manually created transforms: Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


(<torch.utils.data.dataloader.DataLoader at 0x7d873eee7610>,
 ['pizza', 'steak', 'sushi'])

In [27]:
train_dir = image_path / "train"
test_dir = image_path / "test"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

auto_transforms = weights.transforms()
print(f"Auto created transforms: {auto_transforms}")

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=auto_transforms,
    batch_size=32
)
train_dataloader, test_dataloader, class_names

Auto created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x7d87a6a05810>,
 ['pizza', 'steak', 'sushi'])

## 3. Getting a pretrained model

In [28]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights)

In [29]:
for param in model.features.parameters():
  param.requires_grad = False

  set_seeds()

  model.classifier = nn.Sequential(
      nn.Dropout(p=0.2, inplace=True),
      nn.Linear(in_features=1280, out_features=len(class_names), bias=True)
  )


In [30]:
from torchinfo import summary
summary(model, input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20, row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

## 4.Train model and track results

In [31]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [32]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [44]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device:torch.device) -> Dict[str,List]:
  results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader = train_dataloader,
                                       loss_fn = loss_fn,
                                       optimizer = optimizer,
                                       device = device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn = loss_fn,
                                    device = device)

    print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

    writer.add_scalars(main_tag="Loss",
                       tag_scalar_dict={"train_loss":train_loss,
                                        "test_loss":test_loss},
                       global_step=epoch)
    writer.add_scalars(main_tag="Accuracy",
                          tag_scalar_dict={"train_acc": train_acc,
                                          "test_acc": test_acc},
                          global_step=epoch)

    writer.add_graph(model=model,
                     input_to_model=torch.randn(32,3,244,244).to(device))
  writer.close()
  return results



In [39]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [45]:
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.8183 | train_acc: 0.8633 | test_loss: 0.8096 | test_acc: 0.8153
Epoch: 2 | train_loss: 0.8209 | train_acc: 0.6875 | test_loss: 0.7270 | test_acc: 0.8049
Epoch: 3 | train_loss: 0.7220 | train_acc: 0.7383 | test_loss: 0.6093 | test_acc: 0.9167
Epoch: 4 | train_loss: 0.6506 | train_acc: 0.7539 | test_loss: 0.6354 | test_acc: 0.8456
Epoch: 5 | train_loss: 0.5945 | train_acc: 0.7812 | test_loss: 0.6177 | test_acc: 0.8456


In [46]:
results

{'train_loss': [0.8183463215827942,
  0.8209377899765968,
  0.7220034301280975,
  0.6506006456911564,
  0.5945170149207115],
 'train_acc': [0.86328125, 0.6875, 0.73828125, 0.75390625, 0.78125],
 'test_loss': [0.8096047043800354,
  0.7269712487856547,
  0.6093184550603231,
  0.6354186137517294,
  0.6176789800326029],
 'test_acc': [0.8153409090909092,
  0.8049242424242425,
  0.9166666666666666,
  0.8456439393939394,
  0.8456439393939394]}